## Overview

This notebook takes a look into a possible solution to matchmaking in Alpha League.

## General Idea

The total pairing combinations depends on the number of players alive:


| Number of Players Alive | Number of Unique Pairings | Example Lobby |
| ---- | ---| -- |
| 1 | 0 | A |
| 2 | 1 | AB | 
| 3 | 6 | ABC |
| 4 | 3 | ABCD |
| 5 | 60 | ABCDE |
| 6 | 15 | ABCDEF |
| 7 | 630 | ABCDEFG |
| 8 | 105 | ABCDEFGH |

For example, when `4 players` are alive, there are only `3 possible pairings` (i.e. `A` can be paired with `B, C, or D`, so the other pair is always determined). **Note** that ordering isn't considered for now (i.e. home/away boards can be determined later)
```
AB|CD
AC|BD
AD|BC
```

When `3 players` are alive, there are actually more unique pairings (`6`) even though there are less players. After the first 2 players are paired, the `3rd` player can fight either player (see below). The last pairing indicates the ghost fight (second player is the ghost)

```
AB|CA
AB|CB
AC|BA
AC|BC
BC|AB
BC|AC
```

The number of unique pairings is fairly low, so the matchmaking algorithm in this notebook just discovers all the possible matchups, then selects a random matchup every round while following a couple rules:


## Matchmaking Features

* When a player fights another player, they cannot fight them for `X` turns. `X` depends on the number of players alive. This can also determine the "default" fight pool size (`fight_pool = alive_players - self - last_played_players`)

| Num Players Alive | Last X  | Default Fight Pool Size |
| ----------------- | ------- | ----------------------- |
| 8 | 4 | 3 |
| 7 | 4 | 2 |
| 6 | 2 | 3 |
| 5 | 2 | 2 |
| 4 | 2 | 1 |
| 3 | 1 | 1 |

* If there are no valid matchups, the the `last_x` will be temporarily reduced by 1, which should increase the `fight_pool` size and possible # of matchups. This will be referred to as the `last_x-1` edge case in this notebook
 
* There is a list of `ghost_eligible` players which keeps track of players eligible to fight a ghost. Once a player has a fought a ghost, they cannot fight another ghost board until everyone alive has also fought a ghost.
 


## Assumptions

* For simplicity, a player loses when they have reached 8 losses. This can be configured with the `NUM_LIVES` variable
* For simplicity, player names are represented using a single character (e.g. `'A'`)
* There is **always** a winner/loser in each fight. FWIW the matchmaking algorithm shouldn't change much (if at all) from ties.


## Algorithm Overview


- While there is more than 1 player alive:
  - Generate all unique pairing matchups
  - Check if each pairing matchup is `valid`.
    - A `valid` matchup respects all player's fight pools.
      - `fight_pool = alive_players - self - last_x_played_players`
    - If there a ghost fight, the home player must be in `ghost_eligible` for the matchup to be `valid`
  - If there are no valid pairings, continually reduce the `last_x` by `1` until there are valid pairing matchups 
  - For each valid pairing matchup, calculate a `luck` score.
    - `luck` score is currently the # of rounds since they last played the given player
  - Cull the bottom `50%` of "unlucky" matchups (configurable)
  - Select a random matchup
  - Resolve matchups
  
  
## Other Notes

- This is not production quality code, it's kinda messy, redundant, and not very resilient. Apologies if you have to read this version

# TODO

* Come up with a better variable name than `last_x`
* Gracefully handle then `last_x-1` edge case doesn't work
* This notebook currently does not handle home/away boards. It should be easily handled by just randomly assigning home/away boards
* Write tests and discover edge cases for players (e.g. answer questions like "when was the first time a player fought `ABA`/ back-to-back?")
* Check if the `50%` threshold is a "good" threshold


# Ideas for Improvement

* Similar to the `ghost_eligible` players, add a `bad_luck_eligible` player list. Remove the player from the list when they get the short end of the stick (e.g. they fight `ABA` first)
* Implement a feature that forces matchups after some number of rounds. For example, `A` might fight `H` after ~15 rounds

In [ ]:
import logging

import random

from collections import Counter
from itertools import combinations, permutations

from tqdm import tqdm  # only used for QoL

random.seed(420)

In [ ]:
LOGGING_LEVEL = logging.INFO  # logging.DEBUG or logging.INFO
logging.basicConfig(level=LOGGING_LEVEL, format='%(asctime)s %(levelname)s - %(funcName)s: %(message)s')

In [ ]:
# Player config
NUM_LIVES = 8

# Match config
KEEP_TOP_MATCHUPS = 50 # 1 - 100, e.g. keep top 50% of valid matchups, pick randomly after that
MAX_DECREMENT = 2  # throw an error if the last_x decrement exceeds this value 

In [ ]:
def generate_unique_even(chars):
    if len(chars) % 2 != 0:
        raise ValueError('Must be an even number of characters!')

    chars = list(chars)
    num_chars = len(chars)
    all_pairings = set()
    
    unique_pairs = combinations(chars, 2)
    for pairs in combinations(unique_pairs, num_chars // 2):
        used_chars = set()
        valid = True
        sorted_pairs = []
        
        for pair in pairs:
            if pair[0] in used_chars or pair[1] in used_chars:
                valid = False
                break
            
            used_chars.update(pair)
            sorted_pairs.append(tuple(sorted(pair)))
            
        if valid:
            all_pairings.add(tuple(sorted(sorted_pairs)))
    
    return sorted(all_pairings)


def generate_unique_odd(chars):
    if len(chars) % 2 != 1:
        raise ValueError('Must be an odd number of characters!')
        
    
    chars = list(chars)
    num_chars = len(chars)
    all_pairings = []
    
    even_letters_combos = combinations(chars, num_chars-1)
    
    for even_letters in even_letters_combos:
        unique_pairings = generate_unique_even(even_letters)
        last_letter = list(set(chars) - set(even_letters))[0]
        
        for even_pair in unique_pairings:
            
            for char in even_letters:
                pair = (*even_pair, (last_letter, char))
                all_pairings.append(pair)
    
    return all_pairings
        
def generate_unique(chars):
    if len(chars) % 2 == 0:
        return generate_unique_even(chars)
    else:
        return generate_unique_odd(chars)

    
def get_pair_shorthand(pair):
    """Shorten the pairings into a easily digestable string.

    debug purposes only

    >>>pair_shorthand((('A', 'B'), ('C', 'D'), ('E', 'F'), ('G', 'A')))
    'AB|CD|EF|GA'
    """
    shorthands = []
    
    for p in pair:
        shorthands.append(''.join(p))
    
    return '|'.join(shorthands)

In [ ]:
class Player:
    def __init__(self, name):
        self.name = name
        self.past_fights = []
        self.is_ghost_fights = []
        self.past_results = []

        self.num_losses = 0
        
        self.is_killed = False  # used to override/debug
    
    def __repr__(self):
        return f'<Player name={self.name} alive={self.is_alive()}>'
    
    def record_player_fight(self, other_name, is_win, is_ghost=False):       
        self.past_fights.append(other_name)
        self.past_results.append('*' if is_win else ' ')

        if not is_win:
            self.num_losses += 1
        
        self.is_ghost_fights.append(True if is_ghost else False)

    def last(self, n):
        if n == 0:
            # syntax below doesn't work for 0
            return []

        return self.past_fights[-n:]
    
    def can_play(self, other_name, n):
        return other_name not in self.last(n)

    def kill(self):
        # DEBUG ONLY
        self.is_killed = True

    def is_alive(self):
        if self.is_killed:
            return False

        return self.num_losses <  NUM_LIVES
    
    def describe(self):
        """Helper function to list players fights in a single line."""
        is_ghost_string = ['G' if g else ' ' for g in self.is_ghost_fights]
        match_strings = [f'{name}{result}{ghost}' for name, result, ghost in zip(self.past_fights, self.past_results, is_ghost_string)]

        match_string = '>'.join(match_strings)
        top = len(self.past_fights)-self.num_losses
        bottom = len(self.past_fights)
        
        num_ghost_fights = sum(self.is_ghost_fights)
        print(f'Player: {self.name} | Matches ({top:02}/{bottom:02} | {num_ghost_fights:02}): {match_string}')

    def describe_verbose(self):
        match_string = ' > '.join(self.past_fights)
        num_matchups = {}
        for x in sorted(self.past_fights):
            if x in num_matchups:
                num_matchups[x] += 1
            else:
                num_matchups[x] = 1

        print(f'Player: {self.name}')

        print(f'\nMatches: {match_string}')
        for k,v in num_matchups.items():
            print(f'\t{k} : {v}')


In [ ]:
def parse_last_x_rule(rule_string='4321000'):
    if len(rule_string) != 7:
        raise ValueError('rule string must be of len 7')
    split = [int(x) for x in rule_string]
    num_alive = list(range(8, 1, -1))
    return {n:s for n,s in zip(num_alive, split)}


class MatchMaker:
    
    def __init__(self, players, last_x_rule_string='4321000'):
        self.players = players
        self.name2player = {p.name: p for p in self.players}
    
        self.ghost_eligible = set()
        self.reset_ghost_eligible()

        self.last_x_rule = parse_last_x_rule(last_x_rule_string)
        
        self.logger = logging.getLogger()

    @property
    def num_alive(self):
        return sum(1 for p in self.players if p.is_alive())
    
    def get_last_x(self):
        return self.last_x_rule[self.num_alive]

    def reset_ghost_eligible(self):
        self.ghost_eligible = set(self.get_alive_names())
    
    def is_ghost_eligible(self, name):
        return name in self.ghost_eligible
    
    def remove_ghost_eligible(self, name):
        self.ghost_eligible.remove(name)
    
    def update_ghost_eligible(self):
        to_remove = set()
        for name in self.ghost_eligible:
            player = self.name2player[name]
            if not player.is_alive():
                to_remove.add(name)

        for name in to_remove:
            self.remove_ghost_eligible(name)
        
        if len(self.ghost_eligible)==0:
            self.reset_ghost_eligible()

    def get_alive_players(self):
        players = [p for p in self.players if p.is_alive()]
        random.shuffle(players)
        return players

    def get_alive_names(self):
        names = sorted(p.name for p in self.players if p.is_alive())
        return names

    def get_dead_names(self):
        names = sorted(p.name for p in self.players if not p.is_alive())
        return names
    
    def kill_player(self, name):
        # DEBUG ONLY
        self.name2player[name].kill()

    def discover_matchups(self, last_x=None):
        names = self.get_alive_names()
        num_players = len(names)

        self.update_ghost_eligible()
        
        if last_x is None:
            last_x = self.last_x_rule[num_players]

        valid_matchups = []
        unique_matchups = generate_unique(names)  # ALL possible pairings (does not account for home/away)
        
        for matchups in unique_matchups:
            if self.is_valid_matchups(matchups, last_x):
                valid_matchups.append(matchups)

        return valid_matchups


    def is_valid_matchups(self, matchups, last_x):
        """Given the current state of the game, return whether or not a matchup is valid.
        
        Parameters
        ----------
        matchups: 
            (('A', 'B'), ('C', 'D'), ('E', 'F'), ('G', 'A'))
        last_x : int
            Prevent players they've fought in the last X rounds
        
        """
        names = self.get_alive_names()
        num_players = len(names)
        is_last_ghost = True if num_players % 2 == 1 else False

        name2lastplayed = {name:set(self.name2player[name].last(last_x)) for name in names}

        name2fightpool = {}
        for name in names:
            last_played = name2lastplayed[name]
            name2fightpool[name] = set(names) - last_played - set([name])

        # check every matchup besides the last one
        for name1, name2 in matchups[:-1]:
            if name2 not in name2fightpool[name1]:
                return False

            if name1 not in name2fightpool[name2]:
                return False

        # handle last matchup
        name1, name2 = matchups[-1]
        if name2 not in name2fightpool[name1]:
            return False

        if not is_last_ghost and name1 not in name2fightpool[name2]:
            return False
        
        if is_last_ghost and not self.is_ghost_eligible(name1):
            return False
            

        return True

    def calculate_lucky_score(self, matchups):
        """Given a matchup, calculate the 'lucky' score. Higher score is more lucky."""
        names = self.get_alive_names()
        num_players = len(names)
        is_last_ghost = True if num_players % 2 == 1 else False
        
        def _calculate(player, opponent_name):
            score = 0
            for name in reversed(player.past_fights):
                if name == opponent_name:
                    return score

                score += 1

            return score
        
        score = 0
        # score every matchup besides the last one
        for name1, name2 in matchups[:-1]:
            player1 = self.name2player[name1]
            player2 = self.name2player[name2]
            
            score += _calculate(player1, name2)
            score += _calculate(player2, name1)
        
        name1, name2 = matchups[-1]
        player1 = self.name2player[name1]
        score += _calculate(player1, name2)
        
        if not is_last_ghost:
            player2 = self.name2player[name2]
            score += _calculate(player2, name1)

        return score

    def select_top_matchups(self, matchups, perc=KEEP_TOP_MATCHUPS):
        """Select the top percentage of matchups."""
        scores = [self.calculate_lucky_score(m) for m in matchups]
        
        num_to_select = max(1, round(len(matchups) * (perc/100)))
        # Pair elements with scores, sort by score in descending order
        sorted_pairs = sorted(zip(matchups, scores), key=lambda x: x[1], reverse=True)

        # Select the top elements
        top_matchups = [m for m, _ in sorted_pairs[:num_to_select]]
        
        return top_matchups

    def resolve_matchups(self, matchups):
        results = []
        
        last_index = len(matchups) - 1

        names_alive_before_round = self.get_alive_names()
        num_alive_before_round = len(names_alive_before_round)
        is_last_matchup_ghost = num_alive_before_round % 2 == 1
        

        for i, (name_home, name_away) in enumerate(matchups):
            player_home = self.name2player[name_home]
            player_away = self.name2player[name_away]
            
            if i == last_index and is_last_matchup_ghost:
                is_away_ghost = True
            else:
                is_away_ghost = False

            name_winner = random.choice([name_home, name_away])
            is_home_winner = True if name_winner == name_home else False
            player_home.record_player_fight(name_away, is_home_winner, is_away_ghost)
            if is_away_ghost:
                self.remove_ghost_eligible(name_home)
            
            if not is_away_ghost:
                is_away_winner = not is_home_winner
                player_away.record_player_fight(name_home, is_away_winner)
            
            
            result = (name_home, name_away, is_away_ghost, name_winner, len(names_alive_before_round))
            results.append(result)
        
        return results



class Match:
    
    def __init__(self, player_names, last_x_rule_string):
        self.players = [Player(n) for n in player_names]
        self.match_maker = MatchMaker(self.players, last_x_rule_string)

        self.round_data = []
        self.round_number = 1

        self.round_history = []
        self.numalive_history = []
        self.decrement_history = []
        self.decrement_counter = Counter()
        
        self.edgecase_rounds = set()

        self.logger = logging.getLogger()

    def play_match(self):

        while self.match_maker.num_alive > 1:
            self.round_history.append(self.round_number)
            alive_before = set(self.match_maker.get_alive_names())
            self.numalive_history.append(len(alive_before))
            
            matchups = self.match_maker.discover_matchups()
            
            decrement = 0
            while len(matchups) == 0:
                self.edgecase_rounds.add(self.round_number)
                decrement += 1
                if decrement > MAX_DECREMENT:
                    raise ValueError('Something bad happened here. debug!')

                last_x = self.match_maker.get_last_x()
                new_last_x = last_x - decrement
                self.logger.debug('No possible matchups found with last_x @ %s, temporarily reducing by 1', new_last_x)
                
                matchups = self.match_maker.discover_matchups(new_last_x)
            
            self.decrement_history.append(decrement)
            self.decrement_counter[decrement] += 1
            scores = [self.match_maker.calculate_lucky_score(m) for m in matchups]
            self.logger.debug('matchup scores (%s - %s): %s', min(scores), max(scores), scores)
            matchup = random.choice(matchups)
            self.logger.debug('Selecting matchup (out of %s) with %s alive: %s',len(matchups), len(alive_before), get_pair_shorthand(matchup))
            data = self.match_maker.resolve_matchups(matchup)
            self.round_data.append(data)
            
            alive_after = set(self.match_maker.get_alive_names())
            died = alive_before - alive_after
            if died:
                self.logger.debug('Players died %s on round %s', list(died), self.round_number)

            self.round_number += 1

        name_winner = self.match_maker.get_alive_names()[0]
        self.logger.debug('Player wins: %s ', name_winner)
        return name_winner
    
    def describe(self):
        print('Players:')
        print('--------')
        for player in self.players:
            player.describe()

        name_winner = self.match_maker.get_alive_names()[0]
        print(f'\nRounds: {len(self.round_data)}')
        print(f'Winner: {name_winner}')
        print(f'num_retries: {self.match_maker.num_retries}')

    def print_summary(self):
        print('Player    | Fights (* == ghost)')
        
        roundstring = ' '.join([str(x).ljust(2) for x in self.round_history])
        alivestring = ' '.join([str(x).ljust(2) for x in self.numalive_history])
        decstring = ' '.join([str(x).ljust(2) for x in self.decrement_history])
        print(f'Round     | {roundstring}')
        print(f'# Alive   | {alivestring}')
        print(f'Decrement | {decstring}')
        print(f'----------|-----------------------------------------')
        for player in self.players:
            split = []
            for fight_result, is_ghost in zip(player.past_fights, player.is_ghost_fights):
                if is_ghost:
                    split.append(f'{fight_result}*')
                else:
                    split.append(fight_result.ljust(2))
            
            fighthistory = ' '.join(split)

            playerstring = f'{player.name} (A)     ' if player.is_alive() else f'{player.name}         ' 
            fightstring = f'{playerstring}| {fighthistory}'
            print(fightstring)
        
        print(f'\nEdge case rounds: {self.edgecase_rounds}')
    
    def print_fightpool(self):
        names = self.match_maker.get_alive_names()
        if len(names) == 1:
            print('Only 1 person alive')
            return

        last_x_max = self.match_maker.get_last_x()

        for last_x in range(last_x_max, -1, -1):
            print(f'Last X: {last_x}')
            for name in sorted(names):
                player = self.match_maker.name2player[name]
                fightpool = set(names) - set([name]) - set(player.last(last_x))

                print(f'\t{name} - {fightpool}')

In [ ]:
match = Match('ABCDEFGH', '4422210')
match.play_match()
match.print_summary()

## Finding Edge Cases

In [ ]:
# Helper functions to find edge cases

def find_ABA_indexes(fight_history):
    
    indexes = []  # indexes represent the last element
    
    for i in range(len(fight_history) - 2):
        first = fight_history[i]
        second = fight_history[i+1]
        third = fight_history[i+2]
        
        if first == third and first != second:
            indexes.append(i+2)
    
    return indexes

def find_AA_indexes(fight_history):
    indexes = []  # indexes represent the last element
    
    for i in range(len(fight_history) - 1):
        first = fight_history[i]
        second = fight_history[i+1]
        
        if first == second:
            indexes.append(i+1)
    
    return indexes


def find_AAA_indexes(fight_history):
    indexes = []  # indexes represent the last element
    
    for i in range(len(fight_history) - 2):
        first = fight_history[i]
        second = fight_history[i+1]
        third = fight_history[i+2]
        
        if first == second and first==third:
            indexes.append(i+2)

    return indexes


def is_subarray(lst, sub):
    n, m = len(lst), len(sub)
    return any(lst[i:i + m] == sub for i in range(n - m + 1))


def find_fight_history_edge_cases(match):
    counts = {'AA': Counter(), 'AAA': Counter(), 'ABA': Counter()}
    # each sub dictionary is {num_alive (int) : count (int)}
    
    for player in match.players:
        aa_indexes = find_AA_indexes(player.past_fights)
        aaa_indexes = find_AAA_indexes(player.past_fights)
        aba_indexes = find_ABA_indexes(player.past_fights)
        
        aa_numalive = [match.numalive_history[i] for i in aa_indexes]
        aaa_numalive = [match.numalive_history[i] for i in aa_indexes]
        aba_numalive = [match.numalive_history[i] for i in aa_indexes]
        
        counts['AA'].update(aa_numalive)
        counts['AAA'].update(aaa_numalive)
        counts['ABA'].update(aba_numalive)

    # remove cases where there are 2 alive since these are not edge cases
    counts['AA'].pop(2, None)
    counts['AAA'].pop(2, None)
    counts['ABA'].pop(2, None)

    return counts

## Find games

In [ ]:
SAMPLE_SIZE = 1_000

# Find matches where the number of players jump from specific values
# e.g. 6 players > 4 players > 3 players
alive_edge_cases = [
    [6,4,3],
    [8,6,4],
    [7,5,3],
]


alive_matches = []
fighthistory_matches = []
for _ in tqdm(range(SAMPLE_SIZE)):
    match = Match('ABCDEFGH', '4422210')
    match.play_match()
    
    for edge_case in alive_edge_cases:
        if is_subarray(match.numalive_history, edge_case):
            alive_matches.append(match)

    fighthistory_edge_cases = find_fight_history_edge_cases(match)
    if any(fighthistory_edge_cases.values()):
        fighthistory_matches.append(match)

#### Summary

In [ ]:
print(f'{SAMPLE_SIZE} total games')
print(f'{len(alive_matches)} Alive match edge cases')
print(f'{len(fighthistory_matches)} fight history edge cases')

count = 0
for m in fighthistory_matches:
    edge_cases = find_fight_history_edge_cases(m)

    for case,counts in edge_cases.items():
        if any([k>3 for k in counts.keys()]):
            count += 1
            break

print(f'\t {count} cases where ABA happens when there are 4 players or more')

### Print examples

#### Num Alive Edge Cases 

These are not necessarily issues, just defined from the list above.

In [ ]:
for m in alive_matches[:3]:
    m.print_summary()
    print('\n\n')

#### Fight History Edge Cases

edge cases currently includes when there are 3 players alive, so some of these are not concerning

In [ ]:
for m in fighthistory_matches[:3]:
    print(find_fight_history_edge_cases(m))
    m.print_summary()
    print('\n\n')

# Case 1:

Player F dies, so the # of players goes from 5 to 4

If we want to preserve the "can't fight the last 2 players" rule, we hit a "dead state" with the current `last_x=2`

```

Player | Fights (lowercase == ghost fight)		 Fight Pool
A (A)  | B C D E F B C D E F B C D B E 			 {'H'}
B (A)  | A D C F E A D C F E A H C A F 			 {'H', 'E'}
C      | D A B G H D A B G H D A B E 			 
D      | C B A H G C B A H G C E A 			 
E (A)  | F G H A B F G H A B F D H C A 			 {'H', 'B'}
F      | E H G B A E H G B A E c d H B 			 
G      | H E F C D H E F C D H 			 
H (A)  | G F E D C G F E D C G B E F a 			 {'E', 'B'}

```

# Case 2


```
Player | Fights (lowercase == ghost fight)		 Fight Pool
A      | G B C F D G B H C F G B H F e F E c 			 
B      | D A H C E D A G H E D A C G 			 
C      | E D A B H E G F A H E G B E G E F E 			 
D      | B C F G A B H E F G B H E 			 
E (A)  | C F G H B C F D G B C F D C F C A C 			 
F      | H E D A G H E C D A H E G A E A C 			 
G      | A H E D F A C B E D A C F B C 			 
H      | F G B E C F D A B C F D A 			 
```


# Case 3

```
Player | Fights (* == ghost)
Round  | 1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19
# Alive| 8  8  8  8  8  8  8  8  8  8  8  7  7  6  4  3  3  3  3 
-------|-----------------------------------------
A      | G  F  D  E  H  G  F  C  D  H  E 
B (A)  | H  D  G  F  C  H  D  G  F  E  C  H  G  E  H  D  G* G  D 
C      | E  H  F  G  B  E  H  A  G  D  B  F  H  G 
D      | F  B  A  H  G  F  B  E  A  C  F  G  E  H  G  B  G  G* B 
E      | C  G  H  A  F  C  G  D  H  B  A  G* D  B 
F      | D  A  C  B  E  D  A  H  B  G  D  C  E*
G      | A  E  B  C  D  A  E  B  C  F  H  D  B  C  D  B* D  B  D*
H      | B  C  E  D  A  B  C  F  E  A  G  B  C  D  B 
```

# Case 4, 5, and 6

Players dying fast

### 6 > 4 > 3
```
Player | Fights (* == ghost)
Round  | 1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19
# Alive| 8  8  8  8  8  8  8  8  8  8  8  7  7  6  4  3  3  3  2 
-------|-----------------------------------------
A      | D  E  F  G  C  B  H  E  D  C  F  B  G  C 
B (A)  | C  F  E  H  D  A  F  C  G  D  H  A  C  E  C  E  D* E  E 
C      | B  D  H  E  A  F  G  B  H  A  D  G  B  A  B 
D      | A  C  G  F  B  H  E  G  A  B  C  E  F  G  E  B* E  B*
E      | G  A  B  C  H  G  D  A  F  H  G  D  C* B  D  B  D  B  B 
F      | H  B  A  D  G  C  B  H  E  G  A  B* D 
G      | E  H  D  A  F  E  C  D  B  F  E  C  A  D 
H      | F  G  C  B  E  D  A  F  C  E  B 

Edge case rounds: {15}

```

### 7 > 5 > 3
```
Player | Fights (* == ghost)
Round  | 1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18
# Alive| 8  8  8  8  8  8  8  8  8  8  8  7  7  7  5  3  3  3 
-------|-----------------------------------------
A      | F  D  E  G  H  F  D  E  B  G  H  D* E  B  H  C  H  C 
B      | G  H  D  F  C  G  H  D  A  C  F  H  D  A 
C (A)  | D  E  G  H  B  D  E  G  F  B  D  E  H  A* F  A  H* A 
D      | C  A  B  E  G  C  A  B  E  H  C  F  B  E 
E      | H  C  A  D  F  H  C  A  D  F  G  C  A  D  F*
F      | A  G  H  B  E  A  G  H  C  E  B  D  A* H  C 
G      | B  F  C  A  D  B  F  C  H  A  E 
H      | E  B  F  C  A  E  B  F  G  D  A  B  C  F  A  C* A  C*

Edge case rounds: set()

```

### 8 > 6 > 4
```
Player | Fights (* == ghost)
Round  | 1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17
# Alive| 8  8  8  8  8  8  8  8  8  8  8  8  8  6  4  4  3 
-------|-----------------------------------------
A      | E  G  B  C  H  D  E  G  F  H  D  B  G 
B (A)  | C  H  A  F  G  E  D  H  C  G  E  A  H  C  E  D  E 
C      | B  D  E  A  F  G  H  D  B  F  G  E  D  B 
D      | H  C  F  G  E  A  B  C  G  E  A  F  C  E  F  B 
E      | A  F  C  H  D  B  A  F  H  D  B  C  F  D  B  F  B 
F      | G  E  D  B  C  H  G  E  A  C  H  D  E  G  D  E  B*
G      | F  A  H  D  B  C  F  A  D  B  C  H  A  F 
H      | D  B  G  E  A  F  C  B  E  A  F  G  B 

Edge case rounds: set()

```